In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import ktrain
from ktrain import text

In [2]:
data_train = pd.read_excel('movie_reviews/train.xlsx', dtype=str)
data_train.head()

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [3]:
data_test = pd.read_excel('movie_reviews/test.xlsx', dtype=str)
data_test.head()

,Reviews,Sentiment
0,Who would have thought that a movie about a ma...,pos
1,After realizing what is going on around us ......,pos
2,I grew up watching the original Disney Cindere...,neg
3,David Mamet wrote the screenplay and made his ...,pos
4,"Admittedly, I didn't have high expectations of...",neg


In [4]:
(X_train, y_train), (X_test, y_test), preprocess = text.texts_from_df(data_train,
                                                                     text_column='Reviews', 
                                                                     label_columns = 'Sentiment',
                                                                     val_df = data_test,
                                                                     maxlen=100,
                                                                     preprocess_mode='bert')

['neg', 'pos']
   neg  pos
0  1.0  0.0
1  1.0  0.0
2  1.0  0.0
3  1.0  0.0
4  1.0  0.0
['neg', 'pos']
   neg  pos
0  0.0  1.0
1  0.0  1.0
2  1.0  0.0
3  0.0  1.0
4  1.0  0.0
preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [5]:
print(X_train[0].shape)

(25000, 100)


In [6]:
model = text.text_classifier(name='bert',
                            train_data= (X_train, y_train),
                            preproc = preprocess)

Is Multi-Label? False
maxlen is 100
done.


In [7]:
# Get Learning Rate
learner = ktrain.get_learner(model=model,
                            train_data=(X_train, y_train),
                            val_data = (X_test, y_test),
                            batch_size=8)

In [8]:
#learner.lr_find()
#learner.lr_plot()  ## This might takes the day

In [9]:
learner.fit_onecycle(lr=2e-5,epochs=1)



begin training using onecycle policy with max lr of 2e-05...
3125/3125 [==============================] - 1238s 392ms/step - loss: 0.3780 - accuracy: 0.8266 - val_loss: 0.2105 - val_accuracy: 0.9200


In [10]:
learner

In [11]:
predictor = ktrain.get_predictor(learner.model, preproc = preprocess)

In [12]:
data = ['This movie is horrible and the acting really sucks.']

In [13]:
predictor.predict(data)

['neg']

In [14]:
predictor.save('movie-bert')

C:\Users\prave\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [15]:
data = ['what a movie, acting was superb and screenplay,direction was extrordinary']

In [16]:
predictor.predict(data)

['pos']